## The Wonderful World of ML - Session 3 Assignment (Linear Regression)

In addition to the references provided in session 2, I've uploading all the video and course notes from the [**Regression Models**](https://www.coursera.org/learn/regression-models) class from the Johns Hopkins Data Science Specialization on coursera to our meetup repo [here](https://github.com/focods/WonderfulML/tree/master/docs/JHU_DSS_RegMods).  This weeks assignment is adapted from an assignment given in this class.

If you haven't done so by now, install jupyter notebook and configure it with an R kernel if you are an R user.  If you are Python user, your Anaconda install will have Python configured out of the box.  For further details, refer to the top of the session 2 notebooks.

### Only one problem for this session, but it's a good one...

The data for this assignment can be found in our meetup repo:

[https://raw.githubusercontent.com/focods/WonderfulML/master/data/mtcars.csv](https://raw.githubusercontent.com/focods/WonderfulML/master/data/mtcars.csv)

**Here is the scenario:**  *You work for Motor Trend, a magazine about the automobile industry. Looking at a data set of a collection of cars, they are interested in exploring the relationship between a set of variables and miles per gallon (MPG) (outcome). They are particularly interested in the following two questions:*

1. Is an automatic or manual transmission better for MPG?  
2. Quantify the MPG difference between automatic and manual transmissions.

Here is some code to get you started.

In [ ]:
mtcars <- read.csv("https://raw.githubusercontent.com/focods/WonderfulML/master/data/mtcars.csv")

head(mtcars)

Let's do a little EDA...

In [ ]:
suppressMessages(suppressWarnings(library(dplyr)))
suppressMessages(suppressWarnings(library(bindrcpp)))
suppressMessages(suppressWarnings(library(ggplot2)))
library(dplyr)  # make sure you've done an install.packages so you have these
library(ggplot2)

suppressMessages(suppressWarnings(library(GGally)))
suppressMessages(suppressWarnings(library(ggplot2)))
library(GGally) #install.packages("GGally")

func <- function(dat=mtcars, mapping, method="loess", ...) {
    p <- ggplot(data = dat, mapping = mapping)
    p <- p + geom_point() + geom_smooth(method=method, ...)
    
    return(p)
}

g <- ggpairs(mtcars[, 2:ncol(mtcars)], lower = list(continuous = wrap(func, method="lm")),
             title = "Pairs Plot of Variables in mtcars Dataset")
print(g)

Let's take a look at the overall effect of transmission on MPG with this question in mind: *Is an automatic or manual transmission better for MPG?*  

Let's start by taking a look at a box plot comparing all the automatics against all the manual transmissions:

In [ ]:
## PlotTools.R contains multiplot function
source('../../R/PlotTools.R')  # running notebook from docs/solutions/ dir under project

## boxplot to compare automatics vs manuals
p2 <- ggplot(mtcars, aes(factor(am), mpg))
p2 <- p2 + ggtitle("MPG by Transmission Only")
p2 <- p2 + geom_boxplot(aes(fill = factor(am)));
p2 <- p2 + scale_fill_discrete(name="Transmission", breaks=c(0,1),
                               labels=c("Automatics", "Manuals"))
p2 <- p2 + theme(legend.position=c(0.2,0.85))
p2 <- p2 + labs(x = "Transmission (0 = Automatic)", y = "MPG (Miles Per Gallon)")

print(p2)

The plot suggests that manuals get better mileage than automatics.  Let's see if a t-test confirms this:

In [ ]:
t.test(mpg ~ factor(am), paired=FALSE, var.equal=FALSE, data=mtcars)

O.k., if we assume that other factors are **not** confounding our results, then the data suggests that manuals provide give better mileage.  So our next task is to check for factors that may be confounding the results keeping in mind that our goal is to create an *appropriate* linear model which focused on the effects of transmission (**am**), **not** to create a model that best predicts MPG from the dataset.

An *appropriate* model for this goal was considered to be one that accurately quantified the effects of transmission on MPG (**mpg**) with only the required number of variables. Based on our initial assumption, the process of model selection should start by first building a simple linear model with **mpg** as the response and **am** as the regressor to use as a *base*:

In [ ]:
f01 <- lm(mpg ~ am, mtcars); summary(f01)$coef # base model assuming am significant

We can now build successive models in a nested fashion by adding variables to the base model, and evaluating each new addition based on three criteria.  Criterion 1: The t-test from the call to **lm** indicated that the new variable does not flip signs of prior coefficients (indicating interaction between added variable and one of the existing model variables).  Criterion 2: The ANOVA comparing the prior model in the nested chain with the model including the new variable indicated a significant improvement in predicting **mpg** from the prior model.  Criterion 3: If criteria 1 and 2 were met, the confidence intervals of the newly added coefficient for manuals and automatics were calculated and if they did not overlap, this criteria was considered met and the variable included in the model.

In [ ]:
# Add weight
f02 <- lm(mpg ~ am + wt, mtcars); summary(f02)$coef

Transmission flips sign indicating interaction. Signficance of transmission falls substantially when we add weight.  Let's add an interaction term:

In [ ]:
f03 <- lm(mpg ~ am + wt + am*wt, mtcars); summary(f03)$coef

First two criteria are met, so let's look at the ANOVA:

In [ ]:
anova(f02, f03)

Adding **am*wt** interaction term improves mpg prediction, but is it necessary in quantifying the effect of transmission?  Answer this by testing whether the slopes of $\partial{(mpg)} / \partial{(wt)}$ are significantly different for manuals vs. automatics:

In [ ]:
df.manual <- filter(mtcars, am=="1")
df.autom <- filter(mtcars, am=="0")
fit.manual <- lm(mpg ~ wt, data=df.manual)
fit.autom <- lm(mpg ~ wt, data=df.autom)
beta1.hat.manual <- summary(fit.manual)$coef[2, 1]  # -9.084268
beta1.hat.autom <- summary(fit.autom)$coef[2, 1]    # -3.785908
sigma.manual <- summary(fit.manual)$sigma
sigma.autom <- summary(fit.autom)$sigma
sigma.beta1.manual <- sigma.manual / sqrt(sum((df.manual$wt - mean(df.manual$wt))^2))
sigma.beta1.autom <- sigma.autom / sqrt(sum((df.autom$wt - mean(df.autom$wt))^2))
# confidence interval specifics:
t.crit.manual <- qt(0.975, length(df.manual$am) - 2)  # 2.200985
t.crit.autom <- qt(0.975, length(df.autom$am) - 2)  # 2.109816
beta1.int.manual <- beta1.hat.manual + c(-1,1) * t.crit.manual * sigma.beta1.manual
beta1.int.autom <- beta1.hat.autom + c(-1,1) * t.crit.autom * sigma.beta1.autom
# beta1.int.manual: [-11.8500, -6.3186], beta1.int.autom: [-5.4032, -2.1686]

In [ ]:
c(beta1.int.manual, beta1.int.autom)

This tells us that the 95% confidence interval of the slope of **mpg** wrt. **wt** for manual transmissions is [-11.8500, -6.3186] and for automatics is [-5.4032, -2.1686].  Since these don't overlap, these are significant and criteria 3 is met.

Since looking for overlapping confidence intervals is equivalent to doing a t-test on the difference in the two slopes, as a check, let's make sure we get the same result as we just got:

In [ ]:
# t-test specifics:
n.manual <- length(df.manual$am)
n.autom <- length(df.autom$am)
dof <- n.manual + n.autom - 2
s.pooled <- sqrt((((n.manual-1)*sigma.beta1.manual^2) +
                      ((n.autom-1)*sigma.beta1.autom^2)) / dof) # 0.99205
t.stat <- (beta1.hat.manual - beta1.hat.autom) *
          (1/n.manual + 1/n.autom)^0.5 / s.pooled  # 
p_val <- pt(t.stat, dof, lower.tail=TRUE) # 0.0321 consistent w/conf int's: SIG
p_val

Doing it this way is giving us the same results, so this is good news.

$$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\Yv}{\mathbf{Y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\betav}{\mathbf{\beta}}
\newcommand{\gv}{\mathbf{g}}
\newcommand{\Hv}{\mathbf{H}}
\newcommand{\dv}{\mathbf{d}}
\newcommand{\Vv}{\mathbf{V}}
\newcommand{\vv}{\mathbf{v}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\Sv}{\mathbf{S}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\Zv}{\mathbf{Z}}
\newcommand{\Norm}{\mathcal{N}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\grad}{\mathbf{\nabla}}
\newcommand{\ebx}[1]{e^{\wv_{#1}^T \xv_n}}
\newcommand{\eby}[1]{e^{y_{n,#1}}}
\newcommand{\Tiv}{\mathbf{Ti}}
\newcommand{\Fv}{\mathbf{F}}
\newcommand{\ones}[1]{\mathbf{1}_{#1}}
$$

## The Wonderful World of ML - Session 3 Discussion: Logistic Regression

Logistic regression provides a straightforward way to model the probability that one of two mutually exclusive outcomes will occur. This is another way of saying that this is a technique used for **binary classification**. Because this is one of the simplest and most popular classifiers out there, this should be in every machine learning practicioners toolbox.  It also serves as a good starting point for more sophisticated classification techniques such as LDA and QDA which we'll cover in the next session.

What are some examples of binary classification?
+ Will a team win or loss a game?
+ Will a customer buy or not buy?
+ Will a person survive 90 days after a myocardial infarction (heart attack)?
+ Will a passenger survive the Titanic disaster? (kaggle competition)

### What is logistic regression modeling?

At first look, it may seem odd to call this classifier logistic *regression* until you realize that this technique is modeling the underlying conditional probability of an event occuring given one or more predictors.  In other words, we are fitting a function to $p(C=k\,|\, \xv_n)$ where $C=\text{class of }
n^\text{th}\text{ sample}$ which is binary meaning that k can only be 0 or 1.  The term $\xv_n$ is a vector of predictors for the $n^{th}$ sample.

### How is this model constructed?

A natural way to think about modeling a probablity is to divide the count of an event by the total number of events.  Let's start by proposing a function $f(\xv;\wv_k)$ that is proportional to the count or frequency of a given event.  With our $f(\xv;\wv_k)$ as a function of our predictors $\xv$ and some weight parameters $\wv_k$ for each class, we can use this to compute probabilities by dividing $f(\xv;\wv_k)$ for a particular class $k$ by the total count in all classes which can be expressed as:

1.$$
    \begin{align*}
      p(C=k|\xv) = \frac{f(\xv;\wv_k)}{\sum_{m=1}^K f(\xv;\wv_m)} = g_k(\xv)
    \end{align*}
$$

The above ensures that $p(C=k|\xv)$ will be between 0 and 1 if $f(\xv;\wv_k) \gt 0$, but we have another contraint which is:

2.$$
      \begin{align*}
      1  = \sum_{k=1}^K p_k(C=k|\xv) = \sum_{k=1}^K g_k(\xv)
      \end{align*}
$$

If we look closely at this equation 2., we notice that this really gives us only $(k-1)$ contraints on $g_k(\xv)$ because we can always determine one these contraints by subtracting 1 from the sum of the other.  In other words, for some arbitrary $g_m(\xv)$ where $m \ne k$, we can write:

3.$$
      \begin{align*}
      g_m(\xv) = 1 - \sum_{k=1, k \ne m}^K g_k(\xv)
      \end{align*}
$$

So, we'll just set the final $f(\xv;\wv_k)$, for $k=K$, to be 1.  This gives us:

4.$$
      \begin{align*}
        g_k(\xv) = \left \{ \begin{array}{ll}
            \dfrac{f(\xv;\wv_k)}{1+\sum_{m=1}^{K-1} f(\xv;\wv_m)}, & k < K\\
            \dfrac{1}{1+\sum_{m=1}^{K-1} f(\xv;\wv_m)}, & k = K
          \end{array}
        \right .
      \end{align*}
$$

Since we are only considering two classes, we can drop the subscript $k$ and write:

5.$$
      \begin{align*}
        g(\xv) = \dfrac{f(\xv;\wv)}{1+f(\xv;\wv)}
      \end{align*}
$$

So how do we select $f(\xv;\wv)$?  As it turns out, selecting $f(\xv;\wv) = e^{\wv^T \xv}$ allows us to more easily evaluate the gradient of the log-likelihood function which is used to find the best fit parameters which we'll touch on in the next section below.  For now, we'll just point out that selecting our $f(\xv;\wv)$ in this way results in the *logisitic function* shown in equation (4.2) in the ISL and more generally below:

6.$$
      \begin{align*}
        g(\xv) = \dfrac{e^{\wv^T \xv}}{1+e^{\wv^T \xv}} = p(C=k|\xv)
      \end{align*}
$$

If you play with 6. a little, you can easily convince yourself that this is the same function as $sigmoid(x) = \dfrac{1}{1+e^{-\wv^T \xv}}$.  Leaving 6. in the form of the logistic function allows us to more easily see that with a little manipulation, we can rewrite 6. as

7.$$
      \begin{align*}
        \frac{p(C=k|\xv)}{1 - p(C=k|\xv)} = e^{\wv^T \xv} = \text{odds or odds ratio}
      \end{align*}
$$

If we take the natural log of both sides of 7., we get a more general form of what is shown as equation (4.4) in the ISL.

8.$$
      \begin{align*}
        \ln{\Big(\frac{p(C=k|\xv)}{1 - p(C=k|\xv)}\Big)} = \wv^T \xv
      \end{align*}
$$

The left side of 8. is referred to as the *log-odd* or *logit* and we can see clearly that this model assumes that this quantity is linear in the predictors $\xv$.

### How do we find the best parameters for the model?

### What tools are available in R to do logistic regression?

In R, the **glm** function

### How do we handle discrete and continuous variables in our model?